In [1]:
import frontmatter
import glob
import re
from pystac_client import Client
import json
from collections import OrderedDict

import requests
from cognito_client import CognitoClient

In [2]:
collection_ids = [i.split("/")[-1].split(".")[0] for i in glob.glob('ingestion-data//production/collections/*.json')]
collection_ids

['campfire-lst-night-diff',
 'OMI_trno2-COG',
 'lis-global-da-tws',
 'EPA-annual-emissions_1B2b_Natural_Gas_Processing',
 'grdi-shdi-raster',
 'conus-reach',
 'EPA-annual-emissions_1B1a_Coal_Mining_Underground',
 'hls-entropy-difference',
 'snow-projections-median-585',
 'social-vulnerability-index-socioeconomic',
 'EPA-annual-emissions_4B_Manure_Management',
 'geoglam',
 'OMSO2PCA-COG',
 'lis-global-da-tws-trend',
 'EPA-annual-emissions_2B5_Petrochemical_Production',
 'EPA-annual-emissions_1B2b_Natural_Gas_Production',
 'facebook_population_density',
 'lis-tws-nonstationarity-index',
 'social-vulnerability-index-overall',
 'grdi-v1-raster',
 'EPA-annual-emissions_6A_Landfills_Industrial',
 'EPA-annual-emissions_6B_Wastewater_Treatment_Domestic',
 'snow-projections-median-245',
 'landfill-emit',
 'EPA-annual-emissions_6D_Composting',
 'ecco-surface-height-change',
 'hls-l30-002-ej-reprocessed',
 'houston-aod',
 'darnah-flood',
 'IS2SITMOGR4-cog',
 'hls-bais2-v2',
 'lis-global-da-swe',


In [3]:
dataset_paths = glob.glob('datasets/*.mdx')


In [42]:
for dataset in dataset_paths:
    with open(dataset) as f:
        post = frontmatter.load(f)
        try:
            infoDesc = re.split("::markdown",post["infoDescription"])[1].replace('\n', '').strip()
        except:
            pass
        # print(infoDesc)    
        layers = post["layers"]
        
        for layer in layers:
            id = layer["stacCol"]
            try:
                info= layer["info"]
            except:
                info=None
            description = layer["description"] + " \n\n ### Technical Details \n\n " + infoDesc 
            if id in collection_ids:
                
                with open(f'ingestion-data//production/collections/{id}.json', 'r') as f_collection:
                    collection_data = json.load(f_collection)
                    collection_data["description"] = description
                    if info:
                        collection_data["providers"].append({
                            "name": info["source"],
                            "url": None,
                            "roles": [
                                "producer"
                            ]
                        })
                    with open(f'ingestion-data/production/collections_new_metadata/{id}.json', 'w', encoding='utf-8') as file:
                        print(f'writing file: ingestion_data/collections_new_metadata/{id}')
                        json.dump(collection_data, file, ensure_ascii=False, indent=4)

writing file: ingestion_data/collections_new_metadata/barc-thomasfire
writing file: ingestion_data/collections_new_metadata/eis_fire_perimeter
writing file: ingestion_data/collections_new_metadata/houston-ndvi
writing file: ingestion_data/collections_new_metadata/houston-lst-night
writing file: ingestion_data/collections_new_metadata/houston-lst-day
writing file: ingestion_data/collections_new_metadata/houston-landcover
writing file: ingestion_data/collections_new_metadata/houston-lst-diff
writing file: ingestion_data/collections_new_metadata/conus-reach
writing file: ingestion_data/collections_new_metadata/social-vulnerability-index-socioeconomic
writing file: ingestion_data/collections_new_metadata/social-vulnerability-index-socioeconomic-nopop
writing file: ingestion_data/collections_new_metadata/ecco-surface-height-change
writing file: ingestion_data/collections_new_metadata/mtbs-burn-severity
writing file: ingestion_data/collections_new_metadata/hls-bais2-v2
writing file: ingestio

In [34]:
dev_endpoint = "https://staging.openveda.cloud"
dev_client_id = "4rhmpnmnk3rgd9qtiuarllppau"
dev_user_pool_id = "us-west-2_0G3VRilt1"
dev_identity_pool_id = "us-west-2:ad6647b6-b410-4e73-8205-28a066c290fb"

STAC_INGESTOR_API = f"{dev_endpoint}/api/ingest/"

In [35]:
client = CognitoClient(
    client_id=dev_client_id,
    user_pool_id=dev_user_pool_id,
    identity_pool_id=dev_identity_pool_id
)
_ = client.login()

Enter your Cognito username:  saadiq
Enter your Cognito password:  ········
A new password is required. Please provide a new password:  ········


In [36]:
TOKEN = client.access_token

authorization_header = f"Bearer {TOKEN}"
headers = {
    "Authorization": authorization_header,
    "content-type": "application/json",
    "accept": "application/json",
}

In [40]:
collection_new_paths = glob.glob('ingestion-data//production/collections_new_metadata/*.json')

ingest_url = f"{STAC_INGESTOR_API}collections"

for collection_path in collection_new_paths[:1]:
    with open(collection_path, 'r') as f_collection:
        collection_data = json.load(f_collection)
        collection_id = collection_data["id"]
        print(f"posting: {collection_id}")

        response = requests.post(ingest_url, json=collection_data, headers=headers)
        response.raise_for_status()

posting: campfire-lst-night-diff
